In [1]:
import requests
from bs4 import BeautifulSoup as bs
import re
import pandas as pd
import numpy as np

In [2]:
#Verbal Commits as NCAA 2022 Transfer Portal database
url = "https://www.verbalcommits.com/transfers/2022"
r = requests.get(url)
webpage = bs(r.content, features="html.parser")

In [3]:
table = webpage.find("table", attrs={"class", "table full table-hover tablesorter"})
headers = [header.getText() for header in table.find_all("th")]

In [4]:
#first indice was headers
raw_data = table.find_all("tr")[1:]

In [5]:
#collects string stat for each player into a list
data = [[stat.getText() for stat in player.find_all("td")] for player in raw_data]

In [6]:
df = pd.DataFrame(data, columns = headers)
df

,Stars,Position,Name,Class,Ht,Wt,Immediately Eligible,January Eligible,Previous School,New School,Source
0,2,SG,Abee FletcherFletcher Abee,SO,756-3,180,,,The Citadel,UNC Asheville,"Justin Byerly, HoopSeen"
1,2,PG,Abraham HenryHenry Abraham,FR,726-0,175,,,Eastern Illinois,,
2,2,PF,Acliese III LintonLinton Acliese III,RS SR,786-6,235,Yes,,San Francisco State,Eastern Washington,
3,2,SG,Acree KaniKani Acree,RS SO,786-6,185,,,Ball State,,
4,2,PF,Acunzo MattiaMattia Acunzo,RS FR,806-8,225,,,Robert Morris,Youngstown State,"Emiliano Carchia, Sportando"
...,...,...,...,...,...,...,...,...,...,...,...
136,2,PG,Williams ShaunShaun Williams,RS SO,756-3,175,Yes,,Cal State Bakersfield,,
137,2,SF,Witt SaiSai Witt,JR,806-8,230,,,Texas–Rio Grande Valley,,
138,4,PG,Wood ElijahElijah Wood,FR,776-5,175,,,Eastern Illinois,,
139,2,PG,Zambie RyanRyan Zambie,FR,756-3,195,,,Lafayette,,


In [7]:

#changes form of LAST FIRSTFIRST LAST to first-last that corresponds to SRCBB URL
def to_srcbb(name):
    name = re.sub(r'[\.,\']', '', name[int(len(name)/2):].lower())
    
    suffix = re.search("\s(jr|sr|ii|iii|iv)\Z", name)
    if suffix:
        name = name[:suffix.start()] + name[suffix.start() + 1:]
        
    name = name.replace(' ', "-")
    return name
srcbb_name = df['Name'].apply(to_srcbb)
df['Name'] = df['Name'].apply(lambda name: name[int(len(name)/2):])
df['Ht'] = df['Ht'].apply(lambda ht: ht[ht.index("-") - 1:])


In [9]:
transfers = df.loc[df['New School'] == ""].copy().drop(['New School', 'Source'], axis=1)
transfers.insert(3, "SRCBB Name", srcbb_name)

In [10]:
transfers

,Stars,Position,Name,SRCBB Name,Class,Ht,Wt,Immediately Eligible,January Eligible,Previous School
1,2,PG,Henry Abraham,henry-abraham,FR,6-0,175,,,Eastern Illinois
3,2,SG,Kani Acree,kani-acree,RS SO,6-6,185,,,Ball State
6,2,SG,Daniel Begovich,daniel-begovich,SR,6-5,205,,,Stanford
8,2,PG,Troy Boynton,troy-boynton,FR,6-4,175,,,Evansville
10,2,PF,Amir Britt,amir-britt,FR,6-7,210,,,Manhattan
...,...,...,...,...,...,...,...,...,...,...
136,2,PG,Shaun Williams,shaun-williams,RS SO,6-3,175,Yes,,Cal State Bakersfield
137,2,SF,Sai Witt,sai-witt,JR,6-8,230,,,Texas–Rio Grande Valley
138,4,PG,Elijah Wood,elijah-wood,FR,6-5,175,,,Eastern Illinois
139,2,PG,Ryan Zambie,ryan-zambie,FR,6-3,195,,,Lafayette


In [ ]:
srcbb = "https://www.sports-reference.com/cbb/players/" + name +"-1.html"